In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import gmaps

# Import API key
from config import gkey

In [2]:
counties_df = pd.read_csv("2017_Poverty_Data")


In [3]:
counties_df['lat'] = ""
counties_df['lng'] = ""


In [4]:
params = {"key":gkey}
for index, row in counties_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    county_type = row['County']
    state_type = row['State']
    params['address'] = f"{county_type},{state_type}"
    county_lat_lng = requests.get(base_url, params=params).json()
    counties_df.loc[index, "lat"] = county_lat_lng["results"][0]["geometry"]["location"]["lat"]
    counties_df.loc[index, "lng"] = county_lat_lng["results"][0]["geometry"]["location"]["lng"]

In [5]:
counties_df.head()

,State,County,Population,Poverty Count,Poverty White,Poverty Black,Poverty Asian,Poverty Hispanic,Total Poverty Rate (%),lat,lng
0,Iowa,Delaware County,17326.0,1570.0,1492.0,41.0,21.0,111.0,9.061526,42.4634,-91.4435
1,Iowa,Jefferson County,17945.0,2510.0,1601.0,588.0,268.0,155.0,13.987183,41.0236,-91.9099
2,Iowa,Wayne County,6405.0,1001.0,916.0,21.0,3.0,0.0,15.628415,40.6677,-93.3389
3,Iowa,Allamakee County,13940.0,1550.0,1476.0,21.0,11.0,353.0,11.119082,43.2431,-91.4435
4,Iowa,Linn County,220008.0,20280.0,15228.0,3178.0,265.0,1006.0,9.217847,42.066,-91.6761


In [6]:
gmaps.configure(api_key=gkey)

In [7]:
locations = counties_df[["lat", "lng"]].astype(float)

poverty_rate = counties_df["Total Poverty Rate (%)"].astype(float)

In [8]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=poverty_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
heat_layer.dissipating = False
heat_layer.max_intensity = 100
heat_layer.point_radius = 1

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))